In [1]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

데이터 불러오기

In [2]:
data_path= "data"
signal_name = 'signal1'
normal_files = glob.glob(data_path+'/'+signal_name+'/normal/'+"*99.csv")
anormal_drift_files = glob.glob(data_path+'/'+signal_name+'/anormal_drift/'+"*99.csv")
anormal_erratic_files = glob.glob(data_path+'/'+signal_name+'/anormal_erratic/'+"*99.csv")
anormal_hardover_files = glob.glob(data_path+'/'+signal_name+'/anormal_hardover/'+"*99.csv")
anormal_spike_files = glob.glob(data_path+'/'+signal_name+'/anormal_spike/'+"*99.csv")
anormal_stuck_files = glob.glob(data_path+'/'+signal_name+'/anormal_stuck/'+"*99.csv")


In [3]:
print(anormal_drift_files)

['data/signal1/anormal_drift\\signal1_drift99.csv']


In [4]:
def dataReader(path_names):
    data_n = pd.DataFrame() #판다스의 데이터프레임 형태로 프레임 생성
    for i in path_names:
        low_data = pd.read_csv(i)# 판다스 형태로 읽음, 한csv파일씩 읽기 떄문에 다음 라인에서 하나로 합침
        data_n = pd.concat([data_n,low_data],ignore_index=True)
    return data_n

In [5]:
normal_datas = dataReader(normal_files)
drift_datas = dataReader(anormal_drift_files)
hardover_datas = dataReader(anormal_hardover_files)
erratic_datas = dataReader(anormal_erratic_files)
spike_datas = dataReader(anormal_spike_files)
stuck_datas = dataReader(anormal_stuck_files)

In [6]:
all_data = pd.concat([drift_datas,hardover_datas,erratic_datas,spike_datas,stuck_datas],ignore_index=True)

In [7]:
for i in range(len(all_data)):
    if( all_data['error'][i] != 0):
        print(i)
        break

1060


data shape 변경

In [78]:
#입력, 출력값 크기 설정
n_time_in = 16 # 개의 데이터 입력으로 받음
ntime_out = 1 # 다음 한개의 데이터 목표값

In [79]:
#LSTM, LSTM-AE softmax
def X_to_XyLSTM_softmax(data_pd,ntime_in,ntime_out):#X변형할 시계열 데이터 n_time_in 만큼 하나의 input으로 봄, ntime_out만큼 뒤에꺼를 예측
    nsample = len(data_pd) - ntime_in -ntime_out + 1
    X_ntime = [0 for _ in range(nsample)]
    for i in range(nsample):
        X_ntime[i] = data_pd['value'][i:i+ntime_in]
    X_train = np.reshape(X_ntime,(nsample,ntime_in,1))#2차원 배열을 3차원 배열으로
    #print('X',X_train.shape)
    y_nfuture = [0 for _ in range(nsample)]
    for i in range(nsample):
        y_nfuture[i] = data_pd['error'][i+ntime_in:i+ntime_in+ntime_out]
    y_train = np.array(y_nfuture)
    #print('y',y_train.shape)
    return X_train,y_train

In [80]:
X_LSTM_train, y_LSTM_train = X_to_XyLSTM_softmax(all_data,n_time_in,ntime_out)

new_y = []
for i in range(len(y_LSTM_train)):
    if y_LSTM_train[i] == 0:
        new_y.append([1,0,0,0,0,0])
    if y_LSTM_train[i] == 1:
        new_y.append([0,1,0,0,0,0])
    if y_LSTM_train[i] == 2:
        new_y.append([0,0,1,0,0,0])
    if y_LSTM_train[i] == 3:
        new_y.append([0,0,0,1,0,0])
    if y_LSTM_train[i] == 4:
        new_y.append([0,0,0,0,1,0])
    if y_LSTM_train[i] == 5:
        new_y.append([0,0,0,0,0,1])
y_LSTM_train = np.array(new_y)  


In [81]:
print(X_LSTM_train.shape)
print(y_LSTM_train.shape)

(20144, 16, 1)
(20144, 6)


In [82]:
from sklearn.model_selection import train_test_split

# Your dataset (X represents your features, y represents your target variable)
X, y = X_LSTM_train, y_LSTM_train

# Split the data into training and testing sets (80% train, 20% test in this example)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [83]:
"""
#AE
def X_to_XyAE_shape(X,ntime_out): #X변형할 시계열 데이터, ntime_out만큼 뒤에꺼를 예측
    nsample = len(X) - ntime_out
    X_train = np.array(X[:nsample])
    y_train = np.array(X[ntime_out:len(X)])
    #print('X',X_train.shape)
    #print('y',y_train.shape)
    return X_train,y_train

n_time_in = 32 # 10개의 데이터 입력으로 받음
ntime_out = 1 # 다음 한개의 데이터 목표값
X_AE_train, y_AE_train = X_to_XyAE_shape(normal_datas['value'],ntime_out)

"""

"\n#AE\ndef X_to_XyAE_shape(X,ntime_out): #X변형할 시계열 데이터, ntime_out만큼 뒤에꺼를 예측\n    nsample = len(X) - ntime_out\n    X_train = np.array(X[:nsample])\n    y_train = np.array(X[ntime_out:len(X)])\n    #print('X',X_train.shape)\n    #print('y',y_train.shape)\n    return X_train,y_train\n\nn_time_in = 32 # 10개의 데이터 입력으로 받음\nntime_out = 1 # 다음 한개의 데이터 목표값\nX_AE_train, y_AE_train = X_to_XyAE_shape(normal_datas['value'],ntime_out)\n\n"

model 생성

In [84]:
from tensorflow.python.keras.layers import InputLayer, Dense, LSTM
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.python.keras.models import Sequential

#LSTM_AE 모델
LSTM_AE_model = Sequential()

LSTM_AE_model.add(LSTM(128, input_shape=(n_time_in, 1)))
LSTM_AE_model.add(Dense(64, activation='relu',kernel_initializer='random_uniform'))
LSTM_AE_model.add(Dense(32, activation='relu',kernel_initializer='random_uniform'))
LSTM_AE_model.add(Dense(64, activation='relu',kernel_initializer='random_uniform'))
LSTM_AE_model.add(Dense(128, activation='relu',kernel_initializer='random_uniform'))
LSTM_AE_model.add(Dense(6,activation = 'softmax'))

LSTM_AE_model.compile(loss="categorical_crossentropy",optimizer='adam')

#LSTM 모델
LSTM_model = Sequential()

LSTM_model.add(LSTM(128, input_shape=(n_time_in, 1)))
LSTM_model.add(Dense(6,activation = 'softmax')) # error 의 label 이 총 6개 이므로

LSTM_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', Precision(), Recall()])

"""
#AE 모델
AE_model = Sequential()

AE_model.add(InputLayer(input_shape=(1,1)))
AE_model.add(Dense(128, activation='relu',kernel_initializer='random_uniform'))
AE_model.add(Dense(64, activation='relu',kernel_initializer='random_uniform'))
AE_model.add(Dense(32, activation='relu',kernel_initializer='random_uniform'))
AE_model.add(Dense(64, activation='relu',kernel_initializer='random_uniform'))
AE_model.add(Dense(128, activation='relu',kernel_initializer='random_uniform'))
AE_model.add(Dense(ntime_out,activation='linear'))

AE_model.compile(loss="mean_squared_error",optimizer='adam')
"""

'\n#AE 모델\nAE_model = Sequential()\n\nAE_model.add(InputLayer(input_shape=(1,1)))\nAE_model.add(Dense(128, activation=\'relu\',kernel_initializer=\'random_uniform\'))\nAE_model.add(Dense(64, activation=\'relu\',kernel_initializer=\'random_uniform\'))\nAE_model.add(Dense(32, activation=\'relu\',kernel_initializer=\'random_uniform\'))\nAE_model.add(Dense(64, activation=\'relu\',kernel_initializer=\'random_uniform\'))\nAE_model.add(Dense(128, activation=\'relu\',kernel_initializer=\'random_uniform\'))\nAE_model.add(Dense(ntime_out,activation=\'linear\'))\n\nAE_model.compile(loss="mean_squared_error",optimizer=\'adam\')\n'

학습

In [85]:
#LSTM_AE_model.fit(X_LSTM_train, y_LSTM_train, epochs=1, batch_size=100, verbose=0)
history=LSTM_model.fit(X_train,y_train, epochs=1, batch_size=100, verbose=0)
#AE_model.fit(X_AE_train,y_AE_train, epochs=1, batch_size=100, verbose=0)

In [86]:
print(len(X_test))
print(len(y_test))

4029
4029


In [87]:
y_predict = LSTM_model.predict(X_test,verbose=0)

In [88]:
print(y_test[0])
print(y_predict[0])

[0 0 0 0 0 1]
[2.5146406e-03 9.7405842e-05 2.4283314e-04 5.0799747e-04 2.6096407e-04
 9.9637622e-01]


In [89]:
total_predict = []
for data1 in range(len(y_predict)):
    predict_01 = [0 for i in range(6)]
    max_index = np.argmax(y_predict[data1])
    predict_01[max_index] = 1
    total_predict.append(predict_01)
total_predict = np.array(total_predict)

In [90]:
print(y_test[0])
print(total_predict[0])

[0 0 0 0 0 1]
[0 0 0 0 0 1]


In [91]:
cnt = 0
for i in range(len(y_test)):
    result = np.array_equal(y_test[i], total_predict[i]) 
    if(result == False):
        print(i)
        cnt += 1

print("cnt",cnt)
print(len(y_test))

print((len(y_test)-cnt)/len(y_test))

11
16
19
24
31
47
69
83
106
113
120
126
131
151
164
178
195
199
219
229
233
237
242
293
298
303
309
342
349
350
353
371
375
379
387
392
404
405
413
419
428
438
449
479
485
498
543
548
576
578
584
592
651
657
668
709
714
715
719
726
728
744
761
781
786
804
806
810
823
838
847
848
850
855
873
894
913
924
957
996
1011
1012
1019
1027
1033
1045
1048
1050
1101
1123
1125
1132
1154
1156
1168
1175
1207
1229
1257
1272
1282
1304
1311
1338
1344
1359
1361
1381
1383
1395
1434
1437
1443
1456
1475
1478
1479
1483
1488
1491
1505
1508
1509
1536
1541
1551
1558
1571
1573
1580
1595
1605
1635
1637
1639
1646
1652
1656
1673
1685
1688
1711
1712
1721
1722
1730
1738
1755
1761
1763
1776
1784
1801
1840
1849
1855
1872
1874
1884
1892
1896
1915
1921
1927
1935
1942
1943
1945
1972
1973
1998
2013
2018
2051
2052
2106
2127
2133
2147
2148
2149
2155
2188
2195
2211
2254
2280
2311
2327
2329
2331
2334
2356
2361
2455
2511
2513
2519
2546
2568
2579
2597
2604
2614
2617
2628
2638
2666
2679
2682
2717
2724
2750
2766
2830
2855
2863
287

In [92]:
def error_matrix(total_predict,y_test):
    predict_size = len(total_predict)
    #오차행렬 초기화
    tp = 0
    fp = 0
    tn = 0
    fn = 0 
    Accuracy = np.nan
    Recall = np.nan
    Percision = np.nan
    Specificity = np.nan
    for i in range(predict_size):
        modelP = 0
        modelN = 0
        trueP = 0
        trueN = 0
        # 모델에 대한 P,N
        equl =np.array_equal(total_predict[i],np.array([1,0,0,0,0,0]))
        if(equl==True): # 에러가 아니면
            modelN = 1
        else:
            modelP = 1
        #실제값에 대한 P,N
        equl =np.array_equal(y_test[i],np.array([1,0,0,0,0,0]))
        if(equl == True):
            trueN = 1
        else:
            trueP = 1
        #오차 행렬 업데이트
        if(modelP==1 and trueP == 1):
            tp += 1
        elif(modelP == 1 and trueN == 1):
            fp += 1
        elif(modelN == 1 and trueN == 1):
            tn += 1
        elif(modelN == 1 and trueP == 1):
            fn += 1
    if(tp+tn+fp+fn != 0):
        Accuracy = float(tp+tn)/float(tp+tn+fp+fn) # 전체 예측 중 맞게 예측
    if(tp+fn != 0):
        Recall = float(tp)/float(tp+fn) # 실제 오류 중 오류라고 예측한 것
    if(tp+fp != 0):
        Percision = float(tp)/float(tp+fp) # 오류라고 예측한 것 중 실제 오류
    if(fp+tn != 0):
        Specificity = float(tn)/float(fp+tn) # 오류가 아니라고 예측 한 것 중 정말 오류가 아닌 것
    
    return Accuracy,Recall,Percision,Specificity

In [93]:
Accuracy,Recall,Percision,Specificity=error_matrix(total_predict,y_test)

In [94]:
print("n_time_in",n_time_in)
print("=======================")
print("Accuracy",round(Accuracy*100,2))
print("Recall",round(Recall*100,2))
print("Percision",round(Percision*100,2))

print("\n")
print("Specificity",round(Specificity*100,2))

n_time_in 16
Accuracy 95.66
Recall 80.49
Percision 99.86


Specificity 99.97


In [95]:
loss,Accuracy,Precision,Recall = LSTM_model.evaluate(X_test, y_test)
#print(loss)
print("\nn_time_in",n_time_in)
print("=======================")
print("Accuracy",round(Accuracy*100,2))
print("Recall",round(Recall*100,2))
print("Percision",round(Percision*100,2))

126/126 [==============================] - 1s 3ms/step - loss: 0.2430 - accuracy: 0.9213 - precision_1: 0.9246 - recall_1: 0.8178

n_time_in 16
Accuracy 92.13
Recall 81.78
Percision 99.86
